# Problem 1

## `infix_to_prefix(expression)`:

We're used to seeing most mathematical expressions in _infix_ notation, which places the operator between its two operands (e.g. `(A + B)` or in general `(A op B)`). While this may be intuitive for us, it is more difficult for a computer to parse this notation. This is why some programming languages use _prefix_ notation, which places the operator before its operands (e.g. `(+ A B)` or in general `(op A B)`).

Given a `string` representing an expression in _infix_ notation, return a `string` representing the same expression converted to _prefix_ notation.

You may assume that all variables are a single letter and all numbers are non-negative `int`s. You may also assume that the expression is fully parenthesized.

Examples:

* `infix_to_prefix("x")` should return `"x"`
* `infix_to_prefix("(x + 3)")` should return `"(+ x 3)`
* `infix_to_prefix("((3 * x) + (4 * y))")` should return `"(+ (* 3 x) (* 4 y))"`

In [1]:
def infix_to_prefix(expression):
    
    # First, we need a good way to split the string into meaningful parts.
    # If we put spaces around the parens, we can split everything up by whitespace
    tokens = expression.replace("(", " ( ").replace(")", " ) ").split()
    
    # Now we recursively find operands and reorder them
    def swap_expression(index):
        first_token = tokens[index]
        
        # Base case: if no wrapping parens, must be a primitive (var or int)
        if first_token != '(':
            return first_token, index + 1
        
        # Recursive case
        left_operand, op_index = swap_expression(index + 1)
        op = tokens[op_index]
        right_operand, end_index = swap_expression(op_index + 1)
        return f"({op} {left_operand} {right_operand})", end_index + 1
        
    return swap_expression(0)[0]
    

In [2]:
print(infix_to_prefix("x"))
print(infix_to_prefix("(x + 3)"))
print(infix_to_prefix("((3 * x) + (4 * y))"))

x
(+ x 3)
(+ (* 3 x) (* 4 y))


# Problem 2

## `n_bishops(n, bishop_locs, target)`:

Given an `n`x`n` board, a valid placement of bishops on a chess board is one such that no two bishops can attack each other. In chess, bishops can move any distance diagonally. This means that no two bishops can be directly diagonal from each other.

Within the board, there may be bishops already present that **cannot be moved.** The locations of these bishops are stored in `bishop_locs`. The board is oriented such that `(0, 0)` is the cell in the top-left corner and `(0, n-1)` is the cell in the top-right corner.

Given a `target` number of bishops that we want on the board, find a way to place the remaining bishops on the board such that the resulting board is valid and contains `target` number of bishops. Return a set containing the locations of all bishops on the resulting board. In the case that no valid placement of bishops is possible, return `None`.

You may assume that we will only call this with existing set-ups of bishops where no preexisting bishops can attack each other.

**Hint:** Recall from `n_queens` in lecture that given a bishop in `(row1, col1)` and another in `(row2, col2)`, checking whether one can attack the other can be done by simply checking if `abs(row1 - row2) == abs(col1 - col2)`.

Examples:

* If `n = 3`, `bishop_locs = {}`, and `target = 4`, one of many correct assignments is `{(0, 0), (0, 1), (0, 2), (2, 1)}`

![valid_bishop_1](files/valid_bishop_1.png)

* If `n = 3`, `bishop_locs = {(1, 0), (1, 2)}`, and `target = 4`, one of many correct assignments is `{(1, 0), (1, 2), (2, 0), (2, 2)}`. (The black bishops are bishops that cannot be moved)

![valid_bishop_2](files/valid_bishop_2.png)

* If `n = 2`, `bishop_locs = {}`, and `target = 3`, `None` should be returned, because it is impossible to place 3 bishops on a 2x2 board while keeping all bishops safe from attacks from all other bishops.

![invalid_bishop_1](files/invalid_bishop_1.png)

In [3]:
def n_bishops(n, bishop_locs, target):
    # Helper based on the hint above
    def can_attack(bishop1, bishop2):
        row1, col1 = bishop1
        row2, col2 = bishop2
        return abs(row1 - row2) == abs(col1 - col2)
    
    # Base case: we have enough bishops
    if target - len(bishop_locs) == 0:
        return bishop_locs
    
    # Check every square
    for x in range(n):
        for y in range(n):
            new_bishop = (x, y)
            if any(can_attack(old_bishop, new_bishop) for old_bishop in bishop_locs):
                continue

            # If the square is safe, try adding the new bishop
            bishop_locs.add(new_bishop)
            new_bishop_locs = n_bishops(n, bishop_locs, target)
            
            # If adding the new bishop resulted in a correct board, return it
            if new_bishop_locs is not None:
                return new_bishop_locs
            
            # If not, pull it off and try the next square
            bishop_locs.remove(new_bishop)
            
    # If no square is safe, the board is impossible
    return None
    

In [4]:
print(n_bishops(3, set(), 4))
print(n_bishops(3, {(1, 0), (1, 2)}, 4))
print(n_bishops(2, set(), 3))

{(0, 1), (0, 0), (0, 2), (2, 1)}
{(1, 2), (1, 0), (0, 0), (0, 2)}
None


# Problem 3

## `pipe_cutting(requests, stock_length)`:

You are tasked with buying plumbing pipes for a construction project. Your foreman gives you a list of the varying lengths of pipe needed. The local hardware store sells pipes in one fixed length but has a saw for you to cut up the pipes in any way you choose. Your job is to figure out the minimum number of stock pipes required to satisfy the list of requests, in order to save money and minimize waste.

Please implement `pipe_cutting(requests, stock_length)` where `requests` is a list of pipe lengths needed for the project and `stock_length` is the length of the pipes you can purchase at Home Depot. `pipe_cutting` should return an `int` giving the minimum number of purchased pipes needed to satisfy the list of requests.

You can assume that all elements in `requests` are positive numbers that are no longer than the stock pipe length. So, in the worst case, it will take a number of stock pipes equal to the size of the request list. You do not need to report the cutting/division in the optimal solution, just report the minimum number of stock pipes needed.

There are different ways of solving this problem. We recommend, but do not require, a recursive approach for which the hint below will be useful.

**Hint:** For any particular request, the request might be satisfied by choosing a left-over pipe length or cutting from any left-over pipe lengths, or by choosing to buy a new pipe and cutting it to length.

Examples:

* `pipe_cutting([], 8)` returns `0`
* `pipe_cutting([7], 7)` returns `1`
* `pipe_cutting([7, 6, 4], 10)` returns `2` (the cuts are `[7], [6, 4]`)
* `pipe_cutting([4, 3, 4, 1, 7, 8], 10)` returns `3` (the cuts are `[4, 4, 1], [3, 7], [8]`)

In [5]:
def pipe_cutting(requests, stock_length):
    def helper(requests, leftovers):
        # Base case: no requests = no cuts to make
        if not requests:
            return 0
        
        first_request, rest_of_requests = requests[0], requests[1:]
        # If we have the specific request in leftovers, we should always use it
        if first_request in leftovers:
            leftovers.remove(first_request)
            return helper(rest_of_requests, leftovers)
        
        # Otherwise, we should see if using another leftover gives us a better answer
        fewest_cuts = None
        
        for i, leftover in enumerate(leftovers):
            # Can't use leftovers if they're too small
            if leftover < first_request:
                continue
            
            new_leftovers = leftovers[:i] + leftovers[i+1:] + [leftover - first_request]
            cuts = helper(rest_of_requests, new_leftovers)
            if fewest_cuts is None or cuts < fewest_cuts:
                fewest_cuts = cuts
                
        # We should also try using a new one, just in case that works better (or in case we can't use a leftover)
        leftovers.append(stock_length - first_request)
        cuts = helper(rest_of_requests, leftovers) + 1
        if fewest_cuts is None:
            return cuts
        return min(cuts, fewest_cuts)
    
    return helper(requests, [])
                

In [6]:
print(pipe_cutting([], 8))
print(pipe_cutting([7], 7))
print(pipe_cutting([7, 6, 4], 10))
print(pipe_cutting([4, 3, 4, 1, 7, 8], 10))

0
1
2
3
